# 0) Importing libraries and data

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pytorch_lightning as pl

from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
seed_everything(1996, workers=True)

In [ ]:
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")

In [ ]:
train_dir = '../data/train_set.xlsx'
test_dir = '../data/test_set.xlsx'

# 1) Create PL data module

In [ ]:
class NASAPOWERDataModule(pl.LightningDataModule):
    """
    A PyTorch Lightning Data Module for handling NASA POWER data for ET0 prediction.
    
    This class is responsible for preparing and loading the dataset used in the et0 prediction model.
    It includes methods for data preparation, setup, and creating PyTorch DataLoaders for the training, v
    alidation, and test datasets.
    
    Attributes:
        train_dir (str): The file path to the training dataset.
        test_dir (str): The file path to the testing dataset.
        batch_size (int): The size of the batches for the DataLoader.
        train_split (float): The proportion of the data to be used for training.
    
    Methods:
        prepare_data():
            Reads and preprocesses the data from the file paths specified by train_dir and test_dir.
        
        setup(stage=None):
            Prepares the data for training, validation, and testing. This method is responsible for 
            splitting the data and applying any transformations.
        
        train_dataloader():
            Returns a DataLoader for the training dataset.
        
        val_dataloader():
            Returns a DataLoader for the validation dataset.
        
        test_dataloader():
            Returns a DataLoader for the test dataset.
    """
    def __init__(self, train_dir: str, test_dir: str, batch_size=32, train_split=0.7):
        super().__init__()
        self.train_dir = train_dir
        self.test_dir = test_dir
        self.batch_size = batch_size
        self.train_split = train_split

    def prepare_data(self):
        # read, process and split training dataset
        df_train = pd.read_excel(self.train_dir)
        X = df_train.drop('HS', axis=1).values
        y = df_train['HS'].values
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X) # scale features
        X_train, X_val, y_train, y_val = train_test_split(X_scaled, y,
                                                          test_size = 1 - self.train_split,
                                                          random_state=123)
        self.X_train = torch.tensor(X_train, dtype=torch.float32)
        self.y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1)
        self.X_val = torch.tensor(X_val, dtype=torch.float32)
        self.y_val = torch.tensor(y_val, dtype=torch.float32).unsqueeze(-1)

        # read and process testing dataset
        df_test = pd.read_excel(self.test_dir)
        X_test = df_test.drop('ET', axis=1).values
        y_test = df_test['ET'].values
        X_test_scaled = scaler.transform(X_test) # !!!
        self.X_test = torch.tensor(X_test_scaled, dtype=torch.float32)
        self.y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(-1)

    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            self.train_dataset = TensorDataset(self.X_train, self.y_train)
            self.val_dataset = TensorDataset(self.X_val, self.y_val)
        if stage == 'test' or stage is None:
            self.test_dataset = TensorDataset(self.X_test, self.y_test)

    def train_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size,
                          num_workers=1)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size,
                          num_workers=1)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size,
                          num_workers=1)

# 2) Model implementation

In [ ]:
class LitModel(pl.LightningModule):
    """
    A PyTorch Lightning Module for the et0 prediction model using NASA POWER data.

    This class encapsulates the model architecture, training step, validation step, and configuration
      of optimizers. It's designed for easy experimentation with different model architectures and 
      training routines using the PyTorch Lightning framework.

    Attributes:
        model (torch.nn.Module): The neural network model to be trained.
        loss_fn (Callable): The loss function used for training the model.
        lr (float): Learning rate for the optimizer.

    Methods:
        forward(x):
            Defines the forward pass of the model.
        
        training_step(batch, batch_idx):
            Conducts a single training step, including forward pass, loss calculation, and logging.
        
        validation_step(batch, batch_idx):
            Conducts a single validation step, including forward pass and loss calculation.
        
        configure_optimizers():
            Configures the model's optimizers and learning rate scheduler.
    """
    def __init__(self, model, lr=0.01):
        super().__init__()
        self.model = model
        self.learning_rate = lr
        self.test_outputs = []

    def forward(self, x):
        return self.model.forward(x)

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        y_pred = self(x)
        loss = nn.MSELoss()(y_pred, y)

        # check for NaN values
        if torch.isnan(loss):
            print(f"NaN detected at epoch {self.current_epoch}, batch {batch_idx}")

        self.log('train_loss', loss,
                 prog_bar=True, on_step=False,
                 on_epoch=True)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        y_pred = self(x)
        loss = nn.MSELoss()(y_pred, y)
        self.log('val_loss', loss,
                 prog_bar=True, on_step=False,
                 on_epoch=True)
        return loss # !!!

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = nn.MSELoss()(y_pred, y)
        self.test_outputs.append({'y_pred': y_pred.detach(), 'y_true': y.detach()})

        return {'y_pred': y_pred.detach(), 'y_true': y.detach()}

    def on_test_epoch_end(self):
        # Aggregate outputs
        all_outputs = self.all_gather(self.test_outputs)

        # Concatenate all y_pred and y_true from each test step
        y_pred = torch.cat([tmp['y_pred'] for tmp in all_outputs], dim=0)
        y_true = torch.cat([tmp['y_true'] for tmp in all_outputs], dim=0)

        # Convert to numpy arrays for calculation with sklearn
        y_pred_np = y_pred.cpu().numpy()
        y_true_np = y_true.cpu().numpy()

        # Calculate metrics
        r2 = r2_score(y_true_np, y_pred_np)
        rmse = mean_squared_error(y_true_np, y_pred_np, squared=False)
        nrmse = rmse / np.mean(y_true_np)

        # Log metrics
        self.log('test_r2', r2)
        self.log('test_rmse', rmse)
        self.log('test_nrmse', nrmse)

        # store predictions and actual observations
        self.predictions = y_pred_np
        self.actuals = y_true_np

        # store metrics
        self.metrics = {'R2': r2, 'RMSE': rmse, 'nRMSE': nrmse}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(),
                                    self.learning_rate,
                                    weight_decay=0.01)
        return optimizer

In [ ]:
class MLP(nn.Module):
    """
    A multilayer perceptron (MLP) model designed for regression tasks, 
    implemented using PyTorch's neural network module.

    This class defines a simple feedforward neural network with three linear layers, 
    ReLU activations, and dropout for regularization. It's suitable for tasks like 
    predicting continuous variables from a given set of input features.

    Attributes:
        input_size (int): The number of input features the model expects.
        dropout_rate (float): The dropout rate used in the dropout layers for regularization.

    Methods:
        forward(x):
            Defines the forward pass of the model. Takes an input tensor `x` and returns the model's output tensor.

    Example:
        >>> model = MLPModel(input_size=10, dropout_rate=0.5)
        >>> print(model)
        MLPModel(
          (fc1): Linear(in_features=10, out_features=256, bias=True)
          (fc2): Linear(in_features=256, out_features=128, bias=True)
          (fc3): Linear(in_features=128, out_features=1, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.5, inplace=False)
        )
    """
    def __init__(self, input_size=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# 3) Model training

In [ ]:
num_epochs = 100
data_module = NASAPOWERDataModule(train_dir=train_dir, test_dir=test_dir)
mlp_instance = MLP(input_size=10)

In [ ]:
model = LitModel(mlp_instance,
                 lr=0.012022644346174132)
early_stop_callback = EarlyStopping(monitor="val_loss",
                                    min_delta=0.00,
                                    patience=10,
                                    verbose=True,
                                    mode="min")
trainer = pl.Trainer(max_epochs=num_epochs,
                     log_every_n_steps=10,
                     detect_anomaly=False,
                     callbacks=[early_stop_callback])

In [ ]:
# initialize TensorBoard logger
log_name = "vanilla_mlp"
logger = TensorBoardLogger("lightning_logs", name=log_name)

In [ ]:
trainer.fit(model, data_module)

## 3.1) Visualise loss curves

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

# 4) Model testing

In [ ]:
trainer.test(model, data_module)

### 4.2 Plot results

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(model.actuals, model.predictions, alpha=0.5)
plt.xlabel('Predicted ETo (mm/month)', fontsize=18)
plt.ylabel('Observed ETo (mm/month)', fontsize=18)
plt.title('Predicted vs Observed (Testing)', fontsize=18)
plt.plot([min(model.actuals), max(model.actuals)], 
         [min(model.actuals), max(model.actuals)], 
         'r--')

# put metrics in the plot
metrics_text = '\n'.join([f'{key}: {value:.2f}' for key, value in model.metrics.items()])
plt.annotate(metrics_text, xy=(0.05, 0.8), 
             xycoords='axes fraction', 
             bbox=dict(boxstyle="round", 
                       fc="w"))
plt.show()